# Sentiment Analysis using Naive Bayes

In this assignment, we will attempt to label tweets with sentiments (positive, neutral and negative) using Naive Bayes classifier. Naive Bayes is a very basic approach to this problem, but gives surprisingly good accuracy sometimes.

**Fill in the Blanks**

## Importing required libraries

In [3]:
import pandas as pd
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

## Reading dataset

In [4]:
data=pd.read_csv('tweets.csv')
data.drop(data.columns[0],axis=1,inplace=True)
data.dropna(inplace=True)
data

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1
...,...,...
1375,@liberalminds Its trending idiot.. Did you loo...,0
1376,RT @AstoldByBass: #KimKardashiansNextBoyfriend...,0
1377,RT @GatorNation41: gas was $1.92 when Obama to...,1
1378,"@xShwag haha i know im just so smart, i mean y...",1


## Text processing for the tweets

In [5]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
    
def processTweet(tweet):
    # tweet is the text we will pass for preprocessing 
    # convert passed tweet to lower case 

    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    
    # use word_tokenize imported above to tokenize the tweet
    tweet = word_tokenize(tweet)
    return [word for word in tweet if word not in stopwords]

## Process all tweets

In [7]:
processed=[]

for tweet in data['tweets']:
    # process all tweets using processTweet function above - store in variable 'cleaned' 
    cleaned=processTweet(tweet)
    processed.append(' '.join(cleaned))

In [8]:
data['processed'] = processed

In [23]:
data.tail()

,tweets,labels,processed
1375,@liberalminds Its trending idiot.. Did you loo...,0,trending idiot.. look tweets lol making fun ob...
1376,RT @AstoldByBass: #KimKardashiansNextBoyfriend...,0,rt kimkardashiansnextboyfriend barack obama
1377,RT @GatorNation41: gas was $1.92 when Obama to...,1,rt gas 1.92 obama took office ... guess promis...
1378,"@xShwag haha i know im just so smart, i mean y...",1,haha know im smart mean got ta listen obama cu...
1379,#OBAMA: DICTATOR IN TRAINING. If he passes t...,0,obama dictator training passes training course...


## Create pipeline and define parameters for GridSearch

In [47]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2), (1,3)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

## Split data into test and train

In [48]:
# split data into train and test with split as 0.2 
X = data.processed
y = data.labels

x_train, x_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,  
                                                    stratify=y,
                                                    random_state=42)

## Perform classification (using GridSearch)

In [49]:
# perform GridSearch CV with 10 fold CV using pipeline and tuned_paramters defined above 
clf = GridSearchCV(text_clf, tuned_parameters, cv=10)
clf.fit(x_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', MultinomialNB())]),
             param_grid={'clf__alpha': [1, 0.1, 0.01],
                         'tfidf__norm': ('l1', 'l2'),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2), (2, 2), (1, 3)]})

#### Best parameters found on GridSearch

In [50]:
clf.best_params_

{'clf__alpha': 0.01,
 'tfidf__norm': 'l1',
 'tfidf__use_idf': True,
 'vect__ngram_range': (1, 3)}

## Classification report 

In [51]:
# print classification report after predicting on test set with best model obtained in GridSearch
pred_test = clf.predict(x_test)
classification_report(y_test, pred_test)

'              precision    recall  f1-score   support\n\n           0       0.83      0.98      0.90       189\n           1       0.92      0.63      0.75        70\n           2       1.00      0.19      0.32        16\n\n    accuracy                           0.85       275\n   macro avg       0.92      0.60      0.65       275\nweighted avg       0.86      0.85      0.83       275\n'

## Important:

In [52]:
counts = data.labels.value_counts()
print(counts)

0    942
1    352
2     81
Name: labels, dtype: int64


We can see above that the class distribution is highly imbalanced, this would not lead to good sampling of the data for the classifier. For your learning, try using [SMOTE](https://imbalanced-learn.readthedocs.io/en/stable/api.html) to oversample the minority classes and then evaluate the performance with Naive Bayes and compare.

### SMOTE does not work on text data?

In [33]:
from imblearn.over_sampling import SMOTE
from collections import Counter

sm = SMOTE(random_state=42)
x_train_res, y_train_res = sm.fit_resample(x_train.to_numpy(), y_train.to_numpy())
print('Resampled dataset shape {}'.format(Counter(y_train_res)))

ValueError: could not convert string to float: "10 'd prank call obama"

In [ ]:
clf = GridSearchCV(text_clf, tuned_parameters, cv=10)
clf.fit(x_train_res, y_train_res)
pred_test = clf.predict(x_test)
classification_report(y_test, pred_test)